# Test OVH storage

## Module Installation

In [ ]:
#!pip install --user python-swiftclient python-keystoneclient --upgrade

## Connection to server
Using a config file for credentials

In [23]:
import swiftclient
import json
with open("../config.json") as json_data_file:
    config = json.load(json_data_file)
#print(config['swift_storage']['user'])

def swift_con(config):
    user=config['swift_storage']['user']
    key=config['swift_storage']['key']
    auth_url=config['swift_storage']['auth_url']
    tenant_name=config['swift_storage']['tenant_name']
    auth_version=config['swift_storage']['auth_version']
    options = config['swift_storage']['options']
    return swiftclient.Connection(user=user,
                                  key=key,
                                  authurl=auth_url,
                                  os_options=options,
                                  tenant_name=tenant_name,
                                  auth_version=auth_version)

conn = swift_con(config)

In [25]:
#conn.get_account()

## List files

In [4]:
for container in conn.get_account()[1]:
    container_name = container['name']
    print('container_name:', container_name)
for data in conn.get_container(container_name)[1]:
    print('{0}\t{1}\t{2}'.format(data['name'], data['bytes'], data['last_modified']))

container_name: oco2


## Upload files

In [26]:
import glob
import os
for file in glob.glob("/media/data-nvme/dev/datasets/OCO2/csv/oco2_*.xz", recursive=True):
    with open(file, 'rb') as one_file:
            upload_to = '/datasets/oco-2/soudings/'+ os.path.basename(file)
            print('Copy from',file,'to',upload_to)
            conn.put_object(container_name, upload_to,
                                            contents= one_file.read(),
                                            content_type='application/x-xz')    


## Delete files

In [14]:
container_name = 'oco2'
for data in conn.get_container(container_name)[1]:
    file = data['name']
    if 'oco2_1502.csv.xz' in file:
        print('delete', file)
        # UNCOMMENT FOR REAL DELETE conn.delete_object(container_name, file)

delete /datasets/oco-2/soudings/oco2_1502.csv.xz
